# **Analisis Eda del proyecto**

### Apis a utilizar.
Banco Mundial (https://data.worldbank.org)  
Our World in Data (https://ourworldindata.org)  
World Meter (https://worldometer.readthedocs.io/en/latest)  
Datos Macro (https://datosmacro.expansion.com/demografia/natalidad)  
API (https://opengateway.telefonica.com/apis/population-density-data)  
https://www.ine.gob.cl/estadisticas/sociales/censos-de-poblacion-y-vivienda/censo-de-poblacion-y-vivienda

### Recordamos las preguntas de investigacion:

1) ¿Existe relación entre el nivel de urbanización y las tasas de natalidad?  
  
2) ¿Existen patrones regionales o económicos que expliquen diferencias en natalidad? 
  
3) ¿Puede predecirse la tasa de natalidad de un país en base a sus condiciones socioeconómicas? 
  
4) ¿Qué  países  se  comportan  como  outliers  (tienen  tasas  de  natalidad  inusuales  dadas  sus 
características)? 
  
5) ¿Cómo  han  cambiado  las  tasas  de  natalidad  en  las  últimas  dos  décadas,  y  qué  variables  se 
relacionan con esos cambios? 

In [ ]:
# Librerias a importar
from bs4 import BeautifulSoup
import requests
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import seaborn as sns
import json
import time
from os import path

In [ ]:
# Primer intento de creacion de dataframe total
"""
link_banco_mundial_total = "https://data.worldbank.org/indicator/SP.POP.TOTL"
link_banco_mundial_urbana = "https://data.worldbank.org/indicator/SP.URB.TOTL"
link_banco_mundial_fertilidad = "  https://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL?format=json"
datos_macro_natalidad_html = "https://datosmacro.expansion.com/demografia/natalidad"

indicadores = ["SP.POP.TOTL","SP.URB.TOTL","SP.POP.TOTL"]
dataframes = []
# Hacemos la primera solicitud para saber cuántas páginas hay
for i in indicadores:
    url_base = f"http://api.worldbank.org/v2/country/all/indicator/{i}?format=json&per_page=100"
    response = requests.get(url_base)
    data = response.json()
    total_pages = data[0]['pages']
    print(f"Total de páginas: {total_pages}")

    # Lista para guardar todos los datos
    all_data = []

    # Iterar por todas las páginas
    for page in range(1, total_pages + 1):
        response = requests.get(f"{url_base}&page={page}")
        page_data = response.json()[1]  # los datos están en la segunda posición
        all_data.extend(page_data)

    # Normalizamos el JSON y añadimos columna del indicador
    df = pd.json_normalize(all_data)
    df["indicator_id"] = i # <- esto permite identificar de qué indicador viene
    dataframes.append(df)

# Unificamos todos los DataFrames en uno solo
df_total = pd.concat(dataframes, ignore_index=True)
df_total
"""

# Version anterior del bloque de codigo inferior
'''
for i in indicadores:
    print(f"\nDescargando datos de: {i}")
    url_base = f"http://api.worldbank.org/v2/country/all/indicator/{i}?format=json&per_page=100"

    # Primera solicitud para saber cuántas páginas hay
    response = requests.get(url_base)
    
    # SEGUROS PARA QUE NO LANZE ERROR POR ERRORES DE FORMATO EN ALGUNOS DE LOS LINKS INTERMEDIOS
    try:
        data = response.json()
    except Exception:
        print(f"No se pudo decodificar JSON para {i} (respuesta vacía o error del servidor).")
        continue

    if not isinstance(data, list) or len(data) < 2:
        print(f"Sin datos válidos para {i}.")
        continue

    total_pages = data[0]['pages']
    print(f"Total de páginas para {i}: {total_pages}")

    all_data = []

    # Iterar por todas las páginas
    for page in range(1, total_pages + 1):
        url = f"{url_base}&page={page}"
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Error en la página {page} de {i} (HTTP {response.status_code})")
            continue

        #SEGUROS PARA QUE NO LANZE ERROR POR ERRORES DE FORMATO EN ALGUNOS DE LOS LINKS INTERMEDIOS
        try: 
            page_json = response.json()
            if len(page_json) < 2:
                print(f"Página {page} vacía para {i}")
                continue
            page_data = page_json[1]
            all_data.extend(page_data)

        except Exception:
            print(f"Error al decodificar JSON en página {page} de {i}")
            continue

        # Pequeña pausa para evitar sobrecargar el servidor
        time.sleep(0.3)

    # SEGUROS PARA QUE NO LANZE ERROR POR ERRORES DE FORMATO EN ALGUNOS DE LOS LINKS INTERMEDIOS
    if all_data: 
        df = pd.json_normalize(all_data)
        df["indicator_id"] = i
        dataframes.append(df)
        print(f"{len(all_data)} registros añadidos de {i}")
    else:
        print(f"No se añadieron datos para {i}")

# Unificamos todos los DataFrames
if dataframes:
    df_total = pd.concat(dataframes, ignore_index=True)
    print(f"Data unificada con {len(df_total)} registros totales")
else:
    df_total = pd.DataFrame()
    print("No se pudieron descargar datos válidos.")

df_total.head()
'''

In [ ]:
# Nueva funcion para descargar los archivos desde la API de WorldBank
# Ahora deberia de guardar los archivos descargados en un CSV una vez termine
# No la he probado

def descargar_datos_desde_worldbank():
    indicadores = ["SP.POP.TOTL", "SP.URB.TOTL", "SP.DYN.TFRT.IN"]
    dataframes = []
    archivo_csv = "datos_worldbank.csv"

    # Verificar si los datos fueron descargados con anterioridad
    if path.exists(archivo_csv):
        print(f"Archivo {archivo_csv} encontrado. Cargando datos desde el archivo...")
        df_total = pd.read_csv(archivo_csv)
        print(f"Datos cargados exitosamente: {len(df_total)} registros")
        return df_total

    # Si no existe el archivo, descargar los datos
    print("No se encontro archivo.\n"
        "Iniciando descarga de datos desde la API...")
    
    for i in indicadores:
        print(f"\nDescargando datos de: {i}")
        url_base = f"http://api.worldbank.org/v2/country/all/indicator/{i}?format=json&per_page=100"

        # Primera solicitud para saber cuántas páginas hay
        response = requests.get(url_base)
        
        # SEGUROS PARA QUE NO LANZE ERROR POR ERRORES DE FORMATO EN ALGUNOS DE LOS LINKS INTERMEDIOS
        try:
            data = response.json()
        except Exception:
            print(f"No se pudo decodificar JSON para {i} (respuesta vacía o error del servidor).")
            continue

        if not isinstance(data, list) or len(data) < 2:
            print(f"Sin datos válidos para {i}.")
            continue

        total_pages = data[0]['pages']
        print(f"Total de páginas para {i}: {total_pages}")

        all_data = []

        # Iterar por todas las páginas
        for page in range(1, total_pages + 1):
            url = f"{url_base}&page={page}"
            response = requests.get(url)
            if response.status_code != 200:
                print(f"Error en la página {page} de {i} (HTTP {response.status_code})")
                continue

            # SEGUROS PARA QUE NO LANZE ERROR POR ERRORES DE FORMATO EN ALGUNOS DE LOS LINKS INTERMEDIOS
            try: 
                page_json = response.json()
                if len(page_json) < 2:
                    print(f"Página {page} vacía para {i}")
                    continue
                page_data = page_json[1]
                all_data.extend(page_data)

            except Exception:
                print(f"Error al decodificar JSON en página {page} de {i}")
                continue

            # Pequeña pausa para evitar sobrecargar el servidor
            time.sleep(0.3)

        # SEGUROS PARA QUE NO LANZE ERROR POR ERRORES DE FORMATO EN ALGUNOS DE LOS LINKS INTERMEDIOS
        if all_data: 
            df = pd.json_normalize(all_data)
            df["indicator_id"] = i
            dataframes.append(df)
            print(f"{len(all_data)} registros añadidos de {i}")
        else:
            print(f"No se añadieron datos para {i}")

    # Unificamos todos los DataFrames
    if dataframes:
        df_total = pd.concat(dataframes, ignore_index=True)
        print(f"Data unificada con {len(df_total)} registros totales")
        
        # Guardar en CSV
        try:
            df_total.to_csv(archivo_csv, index=False)
            print(f"Datos guardados exitosamente en {archivo_csv}")
        except Exception as e:
            print(f"Error al guardar en CSV: {e}")
    else:
        df_total = pd.DataFrame()
        print("No se pudieron descargar datos válidos.")

    return df_total

SyntaxError: 'continue' not properly in loop (3350692347.py, line 88)

In [ ]:
df_total = descargar_datos_desde_worldbank()
df_total.head()

In [ ]:
df_total.groupby("indicator.value").agg("count")

In [ ]:
# Pivotear el df_total para tener columnas por indicador
df_pivot = df_total.pivot_table(
    index=["country.value", "date"],
    columns="indicator_id",
    values="value"
).reset_index()

# Calcular nivel de urbanización (% de población urbana)
df_pivot["urban_rate"] = (df_pivot["SP.URB.TOTL"] / df_pivot["SP.POP.TOTL"]) * 100

# Limpiar años y convertir valores numéricos
df_pivot["date"] = df_pivot["date"].astype(int)
df_pivot = df_pivot.dropna(subset=["SP.DYN.TFRT.IN", "urban_rate"])

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(
    data=df_pivot,
    x="urban_rate", 
    y="SP.DYN.TFRT.IN", 
    hue='date'
)
plt.title("Relación entre urbanización y tasa de natalidad")
plt.xlabel("% población urbana")
plt.ylabel("Tasa de fertilidad (hijos por mujer)")
plt.show()


In [ ]:
## Posible alternativa de varios graficos dividos en una matriz 3x3.
# 1) Desconozco si los datos realmente llegan desde 1965 al 2010
# 1.1) En caso de ser necesario los rangos pueden ajustarse modificando la lista de tuplas: "year_range"
# 2) No ha sido probado ya que no tengo tiempo de descargar los datos, al tiempo de realizar este commit.

# Crear rangos de años de 5 en 5
year_ranges = [
    (1965, 1969),
    (1970, 1974),
    (1975, 1979),
    (1980, 1984),
    (1985, 1989),
    (1990, 1994),
    (1995, 1999),
    (2000, 2004),
    (2005, 2010)
]

# Crear figura con subgráficos
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.flatten()  # Aplanar para iterar fácilmente

for idx, (start_year, end_year) in enumerate(year_ranges):

    # Filtrar datos para el rango de años actual
    mask = (df_filtered['date'] >= start_year) & (df_filtered['date'] <= end_year)
    df_range = df_filtered.loc[mask]
    
    # Crear scatter plot en el subgráfico correspondiente
    sc = axes[idx].scatter(
        x=df_range["urban_rate"],
        y=df_range["SP.DYN.TFRT.IN"],
        c=df_range["date"],
        cmap="viridis",
        alpha=0.6
    )
    
    # Añadir una colorbar individual para cada subgráfico
    plt.colorbar(sc, ax=axes[idx])
    
    axes[idx].set_title(f"Relación {start_year}-{end_year}")
    axes[idx].set_xlabel("% población urbana")
    axes[idx].set_ylabel("Tasa de fertilidad")

plt.suptitle("Evolución de la relación entre urbanización y tasa de natalidad", fontsize=14, y=0.95)
plt.tight_layout()
plt.show()

In [ ]:
numeric_df = df_pivot.drop(columns=["country_name", "date"]).dropna()
sns.heatmap(numeric_df.corr(), annot=True, cmap="coolwarm")
plt.title("Correlaciones entre variables socioeconómicas y natalidad")
plt.show()


In [ ]:

# Csv DIRECTO our world in data
df = pd.read_csv("https://ourworldindata.org/grapher/distribution-of-population-poverty-thresholds.csv?v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})


In [ ]:
df.info()